# Testing functionality of TKM and JAX

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!export XLA_PYTHON_CLIENT_MEM_FRACTION=.1

In [3]:
import jax
# cpu = jax.devices("cpu")[0]
# jax.default_device(cpu)

In [4]:
# from jax.config import config
# config.update("jax_enable_x64", True)

## Load Banana Data

In [5]:
import pandas as pd
from pathlib import Path

In [6]:
data_path = Path("../data/banana.csv")
df_banana = pd.read_csv(data_path)

In [7]:
df_banana.head()

,V1,V2,Class
0,1.140,-0.114,1
1,-1.520,-1.150,2
2,-1.050,0.720,1
3,-0.916,0.397,2
4,-1.090,0.437,2


In [8]:
# expecting labels 1 and -1

# transform label 2 to -1
df_banana.loc[df_banana.Class == 2, 'Class'] = -1
df_banana.head()

,V1,V2,Class
0,1.140,-0.114,1
1,-1.520,-1.150,-1
2,-1.050,0.720,1
3,-0.916,0.397,-1
4,-1.090,0.437,-1


In [9]:
# standardize

# df_banana[['V1','V2']] = (df_banana[['V1','V2']] -  df_banana[['V1','V2']].min() )/ (df_banana[['V1','V2']].max() - df_banana[['V1','V2']].min())
# df_banana.head()

In [10]:
feature_names = ['V1', 'V2']
x = df_banana[feature_names].to_numpy()

label_name = ['Class']
y = df_banana[label_name].to_numpy()

In [11]:
# rescale

x = (x - x.min(axis=0)) / (x.max(axis=0) - x.min(axis=0))

### Data to JAX array

In [12]:
from jax import numpy as jnp
x = jnp.array(x)
y = jnp.array(y)

In [13]:
from jax import random
key = random.PRNGKey(42)

## Tensor Kernel Machine

### Polynomial

In [27]:
from tkm.model import TensorizedKernelMachine as TKM
from jax import jit
from tkm.features import polynomial

model = TKM(M=12,R=6,feature_map=polynomial)

In [28]:
W = model.fit(key,x,y) #,M=12,R=6,feature_map=polynomial)

In [29]:
%timeit model.fit(key,x,y) #,M=12,R=6,feature_map=polynomial)

7.93 ms ± 847 ns per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [30]:
import jmp

half_policy = jmp.get_policy('half')

model_half = TKM(M=12,R=6,feature_map=polynomial, policy=half_policy)
x_half, y_half = half_policy.cast_to_param((x,y))

model_half.fit(key,x,y)

DeviceArray([[[-1.0057396e+04,  1.5561937e+04, -7.1854185e+03,
                5.6509438e+04, -6.4147301e+04, -5.2884281e+04],
              [ 2.3246876e+01, -5.7960121e+04, -1.4030002e+04,
               -1.5576964e+05,  2.0631802e+05,  1.9556877e+05],
              [ 4.9465035e+04,  2.5154791e+04,  4.7511727e+04,
               -3.1814432e+04, -1.6083208e+04, -5.8015934e+04],
              [-1.9818404e+04,  2.5021057e+04,  2.8232034e+03,
                7.8497039e+04, -8.4285070e+04, -7.5874734e+04],
              [ 1.4082094e+05,  2.4307215e+04,  9.7423570e+04,
                7.6626414e+04, -2.1045316e+05, -1.8981380e+05],
              [-1.3582805e+05, -6.1180047e+04, -1.0620028e+05,
                2.6990828e+03,  1.6952028e+05,  1.7967295e+05],
              [-3.1671088e+04,  5.3115542e+03,  2.8828762e+04,
               -9.0663711e+04,  6.2701000e+04, -4.2626758e+03],
              [ 4.0503379e+04,  4.2725684e+04,  5.4076359e+04,
                5.6085547e+04, -9.4985039e+04, -

In [22]:
fit_polynomial_half = partial(fit, feature_map=polynomial, M=12, R=6, policy=half_policy)

In [24]:
fit_compiled_half = jit(fit_polynomial_half)

In [27]:
%timeit fit_compiled_half(key, x_half, y_half).block_until_ready()

7.87 ms ± 15 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


#### Predict

In [20]:
from tkm.metrics import accuracy

y_hat = model.predict(x, W) #, feature_map=polynomial, M=12,R=6)
acc = accuracy(y.squeeze(), y_hat)
acc

DeviceArray(0.8918868, dtype=float32)

In [21]:
%timeit model.predict(x, W) #, M=12,R=6, feature_map=polynomial)

1.7 ms ± 7.15 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [25]:
# from jax import jit
# from tkm.model import predict_vmap

y_hat = model.predict_vmap(x, W, feature_map=polynomial, M=12, R=6)
acc = accuracy(y.squeeze(), y_hat)
print(acc)

%timeit model.predict_vmap(x, W, feature_map=polynomial, M=12, R=6)

0.8918868
4.21 ms ± 22 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [27]:
from functools import partial

predict_vmap_poly = partial(model.predict_vmap, feature_map=polynomial, M=12, R=6)
predict_vmap_compiled = jit(predict_vmap_poly)

%timeit predict_vmap_compiled(x,W).block_until_ready()

1.7 ms ± 7.71 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


#### fourier features

In [28]:
from tkm.model import TensorizedKernelMachine
from tkm.features import fourier

model_fourier = TensorizedKernelMachine(M=12,R=6,lengthscale=0.5,feature_map=fourier)
W = model_fourier.fit(key,x,y)
y_hat = model_fourier.predict(x, W)
acc = accuracy(y.squeeze(), y_hat)
print(acc)

0.8945283


In [29]:
jax.devices()

[CpuDevice(id=0)]

## batched dotkron

In [31]:
from tkm.model import TensorizedKernelMachine 

model_batch = TensorizedKernelMachine(M=12,R=6,feature_map=polynomial,batch_size=10)
W_batch = model_batch.fit(key,x,y) #,M=12,R=6,feature_map=polynomial)

In [32]:
y_hat = model_batch.predict(x,W_batch)